In [1]:
import boto3,sagemaker,json
from time import sleep

In [2]:
# AWS サービス の API を実行するためのクライアントを生成
transcribe = boto3.client('transcribe')
comprehend = boto3.client('comprehend')
s3 = boto3.client('s3')

In [3]:
# 音声ファイル (sample.mp3) を s3 にアップロード
bucket = sagemaker.session.Session().default_bucket()
mp3file = 'sample.mp3'
input_key_prefix = 'transcribe/input/'
input_key = input_key_prefix+mp3file
s3.upload_file(mp3file,bucket,input_key)
input_s3uri = 's3://'+bucket+'/' + input_key


transcribe_job_name = 'sample'
output_key = 'transcribe/output/'+transcribe_job_name + '.json'

In [4]:
# 音声ファイルを文字起こし
if len(transcribe.list_transcription_jobs(JobNameContains=transcribe_job_name)['TranscriptionJobSummaries'])==1:
    transcribe.delete_transcription_job(TranscriptionJobName=transcribe_job_name)
    sleep(1)
transcribe.start_transcription_job(
    TranscriptionJobName=transcribe_job_name,
    LanguageCode='ja-JP',
    Media={
        'MediaFileUri': input_s3uri
    },
    OutputBucketName=bucket,
    OutputKey=output_key
)

{'TranscriptionJob': {'TranscriptionJobName': 'sample',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'ja-JP',
  'Media': {'MediaFileUri': 's3://sagemaker-ap-northeast-1-377600838340/transcribe/input/sample.mp3'},
  'StartTime': datetime.datetime(2020, 10, 27, 8, 33, 25, 429000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2020, 10, 27, 8, 33, 25, 392000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': 'ad664734-1639-4f9c-9664-aebc581f9da0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 27 Oct 2020 08:33:24 GMT',
   'x-amzn-requestid': 'ad664734-1639-4f9c-9664-aebc581f9da0',
   'content-length': '275',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [5]:
while True:
    status = transcribe.list_transcription_jobs(JobNameContains=transcribe_job_name)['TranscriptionJobSummaries'][0]['TranscriptionJobStatus']
    if status == 'COMPLETED':
        print('.')
        print('COMPLETED!')
        break
    elif status == 'FAILED':
        print('.')
        print('FAILED')
        break
    else:
        print('.',end='')
        sleep(1)

...............
COMPLETED!


In [6]:
s3.download_file(bucket,output_key,'sample.json')

In [7]:
with open('sample.json','rt') as f:
    transcription_json = json.loads(f.read())
transcript_txt = transcription_json['results']['transcripts'][0]['transcript'].replace(' ','').replace('、','')
print(transcript_txt)

私は今日初めて自然言語処理サービスを利用して通話内容を可視化します


In [8]:
comprehend.detect_dominant_language(Text = transcript_txt)

{'Languages': [{'LanguageCode': 'ja', 'Score': 0.9989801645278931}],
 'ResponseMetadata': {'RequestId': 'b153a6fc-e563-4e6e-9caf-5e1e3b0961d7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b153a6fc-e563-4e6e-9caf-5e1e3b0961d7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '64',
   'date': 'Tue, 27 Oct 2020 08:33:39 GMT'},
  'RetryAttempts': 0}}

In [9]:
comprehend.detect_entities(Text = transcript_txt, LanguageCode= 'ja')

{'Entities': [{'Score': 0.9712674617767334,
   'Type': 'DATE',
   'Text': '今日',
   'BeginOffset': 2,
   'EndOffset': 4}],
 'ResponseMetadata': {'RequestId': '8453498a-a13d-4b51-8582-f3a131ea3275',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8453498a-a13d-4b51-8582-f3a131ea3275',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Tue, 27 Oct 2020 08:33:40 GMT'},
  'RetryAttempts': 0}}

In [10]:
comprehend.detect_key_phrases(Text = transcript_txt, LanguageCode='ja')

{'KeyPhrases': [{'Score': 1.0, 'Text': '私', 'BeginOffset': 0, 'EndOffset': 1},
  {'Score': 0.9996367692947388,
   'Text': '今日',
   'BeginOffset': 2,
   'EndOffset': 4},
  {'Score': 0.9994778037071228,
   'Text': '自然言語処理サービス',
   'BeginOffset': 7,
   'EndOffset': 17},
  {'Score': 1.0, 'Text': '通話内容', 'BeginOffset': 22, 'EndOffset': 26}],
 'ResponseMetadata': {'RequestId': '93a7565c-000b-42c4-8eee-99ade6431ca8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '93a7565c-000b-42c4-8eee-99ade6431ca8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '316',
   'date': 'Tue, 27 Oct 2020 08:33:40 GMT'},
  'RetryAttempts': 0}}

In [11]:
comprehend.detect_sentiment(Text = transcript_txt, LanguageCode = 'ja')

{'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Positive': 0.00765804760158062,
  'Negative': 6.318943633232266e-05,
  'Neutral': 0.992274820804596,
  'Mixed': 3.848530923278304e-06},
 'ResponseMetadata': {'RequestId': '1a626db6-8413-49ba-a04c-920bf3d11080',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1a626db6-8413-49ba-a04c-920bf3d11080',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '162',
   'date': 'Tue, 27 Oct 2020 08:33:40 GMT'},
  'RetryAttempts': 0}}

In [12]:
transcription_json['results']['transcripts'][0]['transcript']

'私 は 今日 初めて 自然 言語 処理 サービス を 利用 し て 通話 内容 を 可視 化 し、 ます'